In [1]:
import os
#os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

os.environ['TF_USE_LEGACY_KERAS'] = '1';

In [2]:
input_default_json_conf_file='fcnn_emotion4_training_default.json';

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import json

2024-07-24 03:39:25.860614: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 03:39:27.639809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import sys
sys.path.append('../library');

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-07-24 03:39:29.833354: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 03:39:30.064104: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 03:39:30.064491: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
## Load json conf json file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [7]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_base_dir    = DATA['dataset_train_base_dir'];
dataset_labels_file = DATA['dataset_train_labels_file'];

dataset_base_test_dir    = DATA['dataset_test_base_dir'];
dataset_labels_test_file = DATA['dataset_test_labels_file'];

dataset_name        = DATA['dataset_name'];

## Training hyperparameters
EPOCAS     = DATA["epochs"];
BATCH_SIZE = DATA["batch_size"];


## Output
output_base_dir = DATA["output_base_dir"];


NCOD=15;
ANGLE=60;

# Parametros de entrada

In [8]:
for n in range(len(sys.argv)):
    if sys.argv[n]=='--dataset-train-dir':
        dataset_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-train-file':
        dataset_labels_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-test-dir':
        dataset_base_test_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-test-file':
        dataset_labels_test_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--epochs':
        EPOCAS=int(sys.argv[n+1]);
    elif sys.argv[n]=='--batch-size':
        BATCH_SIZE=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
        
print('        dataset_base_dir:',dataset_base_dir)
print('     dataset_labels_file:',dataset_labels_file)
print('   dataset_base_test_dir:',dataset_base_test_dir)
print('dataset_labels_test_file:',dataset_labels_test_file)
print('            dataset_name:',dataset_name)
print('                  EPOCAS:',EPOCAS)
print('              BATCH_SIZE:',BATCH_SIZE)
print('         output_base_dir:',output_base_dir)

        dataset_base_dir: /media/fernando/Expansion/DATASET/TESE/BER/BER2024/BER2024-SKELETON
     dataset_labels_file: train.csv
   dataset_base_test_dir: /media/fernando/Expansion/DATASET/TESE/BER/BER2024/BER2024-SKELETON
dataset_labels_test_file: test.csv
            dataset_name: ber2024-skel
                  EPOCAS: 1000
              BATCH_SIZE: 1024
         output_base_dir: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4


# Set seed of random variables


In [9]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [10]:
# Load filenames and labels
train_val_df = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
#print(train_val_df)

# Setting labels
Y = train_val_df[['label']];
L=np.shape(Y)[0];

# Load test filenames and labels
test_df = pd.read_csv(os.path.join(dataset_base_test_dir,dataset_labels_test_file));

print('\n\ntest_df')
print(test_df)




test_df
              d0         d1        d2         d3         d4        d5  \
0      578.53033  230.53235  0.997254  584.17610  221.87859  0.972028   
1      920.09784  375.53006  0.972105  944.41710  346.81830  0.997568   
2      766.30960  252.15428  0.994588  777.61940  225.52296  0.991696   
3      710.24036  232.16788  0.993728  724.85980  201.97440  0.998824   
4      743.05970  192.98499  0.992488  765.76780  169.78462  0.989598   
...          ...        ...       ...        ...        ...       ...   
16674  439.34560  229.46915  0.991680  457.49250  200.63686  0.990643   
16675  831.20184  257.23987  0.993503  870.40430  225.87146  0.991977   
16676  675.22986  146.00938  0.998339  714.07970  129.44043  0.990368   
16677  365.72820  188.31245  0.993100  383.71097  165.13180  0.991528   
16678  820.39874  276.85623  0.992258  846.90924  255.66278  0.990174   

              d6          d7        d8          d9  ...  d42  d43  d44  d45  \
0      555.77313  209.135130  0.98

# Setting the cross-validation split


In [11]:
from sklearn.model_selection import train_test_split

training_df, validation_df = train_test_split(train_val_df, test_size=0.2,shuffle=True, stratify=Y)

print('\n\ntraining_df')
print(training_df);


print('\n\nvalidation_df')
print(validation_df)



training_df
               d0         d1        d2          d3         d4        d5  \
8982    960.05600  202.20062  0.994441   976.69696  189.05788  0.992597   
11571   734.76117  259.07974  0.993402   759.39734  234.89705  0.991606   
17123   652.60890  174.34769  0.992906   675.89560  160.28514  0.989038   
20066   556.48460  194.43310  0.998464   589.38180  167.67004  0.993131   
8822    429.19846  220.50840  0.992251   463.33950  198.33228  0.998347   
...           ...        ...       ...         ...        ...       ...   
7149    490.36950  226.31702  0.988921   496.08840  212.05159  0.986984   
20120  1056.60690  265.03806  0.990833  1076.93210  245.03746  0.986916   
14275   476.07532  332.64615  0.992961   495.98813  290.08920  0.993253   
5284    570.50950  168.71097  0.992591   592.79803  150.86066  0.990587   
7290    746.79913  177.09557  0.990950   764.65170  162.77843  0.989897   

               d6         d7        d8         d9  ...  d42  d43  d44  d45  \
8982   

# Data augmentation configuration

In [12]:
import SkeletonEmotion4Lib.DataAugmentation as sda
import SkeletonEmotion4Lib.lib_tools as slt

training_data_array   = slt.batch_normalize_coordinates(training_df.iloc[:,0:51].values)
validation_data_array = slt.batch_normalize_coordinates(validation_df.iloc[:,0:51].values)
test_data_array       = slt.batch_normalize_coordinates(test_df.iloc[:,0:51].values)

print('train:',training_data_array.shape)
print('val:',validation_data_array.shape)
print('test:',test_data_array.shape)

train_data_generator = sda.DataAugmentationEncDecGenerator(training_data_array, 
                                    batch_size=BATCH_SIZE, 
                                    augment_fn=lambda X: slt.batch_random_rotate_coordinates(X,angle=ANGLE));

valid_data_generator = sda.DataAugmentationEncDecGenerator(validation_data_array, 
                                        batch_size=BATCH_SIZE, 
                                        augment_fn=None);

test_data_generator = sda.DataAugmentationEncDecGenerator(   test_data_array, 
                                            batch_size=BATCH_SIZE, 
                                            augment_fn=None);




train: (21594, 51)
val: (5399, 51)
test: (16679, 51)


# Creating output directory

In [13]:
output_dir = os.path.join(output_base_dir,dataset_name,'training_validation_holdout');

os.makedirs(output_base_dir,exist_ok = True);

os.makedirs(output_dir,exist_ok = True);

# Create new encoder-decoder model

In [14]:
import SkeletonEmotion4Lib.lib_model as mpp

model = mpp.create_model_encdec(load_weights=False,file_of_weight='',ncod=NCOD);

print('input_shape',model.input_shape)
print('output_shape',model.output_shape)
print('')

model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats_endec.m'));


input_shape (None, 51)
output_shape (None, 51)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 15)                17128     
                                                                 
 sequential_1 (Sequential)   (None, 51)                17164     
                                                                 
Total params: 34292 (133.95 KB)
Trainable params: 34292 (133.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-07-24 03:39:37.538711: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 03:39:37.538936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 03:39:37.539090: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [15]:
# COMPILE NEW MODEL
model.compile(loss='mse',
              optimizer='adam',
              metrics=['RootMeanSquaredError'])

# CREATE CALLBACKS
best_model_encdec_file=os.path.join(output_dir,'model_encdec.h5');
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_encdec_file, 
                                                save_weights_only=True,
                                                monitor='val_loss', 
                                                save_best_only=True, 
                                                verbose=1);

# Definindo o callback EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(  monitor='val_loss', 
                                                    patience=max(10,int(EPOCAS/10)),  
                                                    verbose=1, 
                                                    restore_best_weights=False);

log_dir = os.path.join(output_dir,"logs","fit",'encdec-'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"));
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# FIT THE MODEL
history = model.fit(train_data_generator,
                    epochs=EPOCAS,
                    validation_data=valid_data_generator,
                    callbacks=[checkpoint,early_stopping,tensorboard_callback],
                    verbose=1
                   );

Epoch 1/1000


I0000 00:00:1721803191.993032  164347 service.cc:145] XLA service 0x75e38c208100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721803191.993065  164347 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2024-07-24 03:39:51.999252: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-24 03:39:52.017423: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1721803192.096888  164347 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


21/22 [===========================>..] - ETA: 0s - loss: 0.6036 - root_mean_squared_error: 0.7769
Epoch 1: val_loss improved from inf to 0.47159, saving model to /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4/ber2024-skel/training_validation_holdout/model_encdec.h5
22/22 [==============================] - 16s 77ms/step - loss: 0.5982 - root_mean_squared_error: 0.7734 - val_loss: 0.4716 - val_root_mean_squared_error: 0.6867
Epoch 2/1000
22/22 [==============================] - ETA: 0s - loss: 0.4128 - root_mean_squared_error: 0.6425
Epoch 2: val_loss improved from 0.47159 to 0.31761, saving model to /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4/ber2024-skel/training_validation_holdout/model_encdec.h5
22/22 [==============================] - 1s 39ms/step - loss: 0.4128 - root_mean_squared_error: 0.6425 - val_loss: 0.3176 - val_root_mean_squared_error: 0.5636
Epoch 3/1000
22/22 [==============================] - ETA: 0s - loss: 0.3548 - root_mean_squared_error: 0.

In [ ]:
mpp.save_model_history(history,
                       os.path.join(output_dir,"historical_encdec.csv"),
                       show=False,
                       labels=['root_mean_squared_error','loss']);

In [ ]:
model.load_weights(best_model_encdec_file); 


best_model_encoder_file=os.path.join(output_dir,'model_encoder.h5');
best_model_decoder_file=os.path.join(output_dir,'model_decoder.h5');

model.layers[0].save_weights(best_model_encoder_file);
model.layers[1].save_weights(best_model_decoder_file);

# Evaluate best encode-decoder model

In [ ]:
# LOAD BEST MODEL to evaluate the performance of the model
model.load_weights(best_model_encdec_file);
data_results=dict();

# Evaluate training
results = model.evaluate(train_data_generator)
results = dict(zip(model.metrics_names,results))
print('training',results,"\n\n");
for key,value in results.items():
    data_results['train_'+key]=value;

# Evaluate validation
results = model.evaluate(valid_data_generator)
results = dict(zip(model.metrics_names,results))
print('validation',results,"\n\n");
for key,value in results.items():
    data_results['val_'+key]=value;

# Evaluate testing
results = model.evaluate(test_data_generator)
results = dict(zip(model.metrics_names,results))
print('testing',results,"\n\n");
for key,value in results.items():
    data_results['test_'+key]=value;

data_results['number_of_parameters']=mpp.get_model_parameters(model);

# final all json
with open(os.path.join(output_dir,"training_data_results_encdec.json"), 'w') as f:
    json.dump(data_results, f,indent=4);

tf.keras.backend.clear_session()


# Dataset

In [ ]:

training_dataset = sda.DataAugmentationClsGenerator(training_data_array, 
                                                    training_df.iloc[:,51].values,
                                                    batch_size=BATCH_SIZE, 
                                                    augment_fn=lambda X: slt.batch_random_rotate_coordinates(X,angle=ANGLE));

validation_dataset = sda.DataAugmentationClsGenerator(  validation_data_array, 
                                                        validation_df.iloc[:,51].values,
                                                        batch_size=BATCH_SIZE, 
                                                        augment_fn=None);

test_dataset = sda.DataAugmentationClsGenerator(test_data_array, 
                                                test_df.iloc[:,51].values,
                                                batch_size=BATCH_SIZE, 
                                                augment_fn=None);

# Create new model

In [ ]:

model = mpp.create_model(load_weights=False,
                         file_of_weight=best_model_encoder_file,
                         file_of_weight_full=False,
                         ncod=NCOD);

print('input_shape',model.input_shape)
print('output_shape',model.output_shape)
print('')

model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats_cls.m'));

In [ ]:
# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

# CREATE CALLBACKS
best_model_file=os.path.join(output_dir,'model_cls.h5');
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_file, 
                                                save_weights_only=True,
                                                monitor='val_loss', 
                                                save_best_only=True, 
                                                verbose=1);

# Definindo o callback EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(  monitor='val_loss', 
                                                    patience=max(10,int(EPOCAS/10)),  
                                                    verbose=1, 
                                                    restore_best_weights=False);

log_dir = os.path.join(output_dir,"logs","fit",'cls-'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"));
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# FIT THE MODEL
history = model.fit(training_dataset,
                    epochs=EPOCAS,
                    validation_data=validation_dataset,
                    callbacks=[checkpoint,early_stopping,tensorboard_callback],
                    verbose=1
                   );

In [ ]:
mpp.save_model_history(history,
                       os.path.join(output_dir,"historical_cls.csv"),
                       show=False,
                       labels=['categorical_accuracy','loss']);

In [ ]:
# LOAD BEST MODEL to evaluate the performance of the model
model.load_weights(best_model_file);
data_results=dict();

# Evaluate training
results = model.evaluate(training_dataset)
results = dict(zip(model.metrics_names,results))
print('training',results,"\n\n");
for key,value in results.items():
    data_results['train_'+key]=value;

# Evaluate validation
results = model.evaluate(validation_dataset)
results = dict(zip(model.metrics_names,results))
print('validation',results,"\n\n");
for key,value in results.items():
    data_results['val_'+key]=value;

# Evaluate testing
results = model.evaluate(test_dataset)
results = dict(zip(model.metrics_names,results))
print('testing',results,"\n\n");
for key,value in results.items():
    data_results['test_'+key]=value;

data_results['number_of_parameters']=mpp.get_model_parameters(model);

# final all json
with open(os.path.join(output_dir,"training_data_results_cls.json"), 'w') as f:
    json.dump(data_results, f,indent=4);

tf.keras.backend.clear_session()